In [ ]:
!ls /workspace/stable-diffusion-webui/models/Stable-diffusion

In [ ]:
"""Threads

The easy Python threading module.
"""
from functools import wraps

try:
    import threading
except ImportError:
    import dummy_threading as threading


class Empty(object):
    """An empty value.

    ``Empty`` is a uniquely identifiable object that signifies a ``Channel`` is
    empty. ``False`` or ``None`` are inappropriate for this purpose since those
    values could be sent through the ``Channel``.
    """

    def __repr__(self):
        return "<Empty>"


empty = Empty()  # The empty value.


class _Queue(object):
    """Simple wrap-around queue.

    :param int size: The size of the queue.
    """

    def __init__(self, size=1):
        self._queue = [empty for _ in range(size)]
        self._start = self._end = 0

    def __getitem__(self, key):
        return self._queue[key]

    def __setitem__(self, key, value):
        self._queue[key] = value

    def __len__(self):
        return len(self._queue)

    @property
    def empty(self):
        """Return bool indicating if queue is empty."""
        return self._queue[self._start] is empty

    @property
    def full(self):
        """Return bool indicating if queue is full."""
        return self[self._end] is not empty

    def _inc_index(self, index):
        """Properly increment the index and return."""
        return (index + 1) % len(self)

    def pop(self):
        """Pop a value from the front of the queue.

        :raises ValueError: When the queue is empty.
        """
        if self.empty:
            # This should never happen if the queue is managed properly.
            raise ValueError("Cannot pop from empty queue.")

        val = self[self._start]

        self[self._start] = empty
        self._start = self._inc_index(self._start)

        return val

    def push(self, value):
        """Push the value onto the back of the queue.

        :raises ValueError: When the queue is full.
        """
        if self.full:
            # This should never happen if the queue is managed properly.
            raise ValueError("Cannot push onto full queue.")

        self[self._end] = value
        self._end = self._inc_index(self._end)


class Channel(object):
    """A thread-safe channel to send data through.

    :param int size: The size of the channels buffer.
    """

    def __init__(self, size=1):
        self._values = _Queue(size)
        self._lock = threading.Lock()
        self._recv_val = threading.Condition(self._lock)
        self._send_val = threading.Condition(self._lock)

    def send(self, value):
        """Send a value through the channel.

        If the channel is full, sends the active thread to sleep.
        """
        with self._send_val:
            while self._values.full:
                # Wait for the channel to open up.
                self._send_val.wait()

            self._values.push(value)

            # Notify the ``recv_val`` condition that the channel has a value.
            self._recv_val.notify()

    def recv(self):
        """Receive a value from the channel.

        If no value is available, sends the active thread to sleep.
        """
        with self._recv_val:
            while self._values.empty:
                # Wait for a value to be sent through the channel.
                self._recv_val.wait()

            val = self._values.pop()

            # Notify the ``send_val`` condition that the channel is open.
            self._send_val.notify()

            return val


def threaded(f):
    """Create a threaded function.

    :param f: A function.
    :returns: A ``threading.Thread`` object that the function is running in.

    :example:
        @threaded
        def threaded_add(x, y):
            x + y

        threaded_add(1, 2)  # This runs in a new thread!

    :example:
        def add(x, y):
            x + y

        threaded_add = threaded(add)

        add(1, 2)           # This runs in the main thread.
        threaded_add(1, 2)  # This runs in a new thread!
    """

    @wraps(f)
    def threaded_wrapper(*args, **kwargs):
        t = threading.Thread(target=f, args=args, kwargs=kwargs)
        t.start()
        return t

    return threaded_wrapper


In [ ]:
import subprocess

@threaded
def download_model(url):
    command = ['wget', url, '--content-disposition', '--no-verbose', '--no-clobber', '-P', '/workspace/stable-diffusion-webui/models/Stable-diffusion/']
    subprocess.run(command)

    
# https://civitai.com/api/download/models/87153
@threaded
def download_lora(url):
    command = ['wget', url, '--content-disposition', '--no-verbose', '--no-clobber', '-P', '/workspace/stable-diffusion-webui/models/Lora/']
    subprocess.run(command)

@threaded
def download_vae(url):
    command = ['wget', url, '--content-disposition', '--no-verbose', '--no-clobber', '-P', '/workspace/stable-diffusion-webui/models/VAE/']
    subprocess.run(command)

@threaded
def download_embedding(url):
    command = ['wget', url, '--content-disposition', '--no-verbose', '--no-clobber', '-P', '/workspace/stable-diffusion-webui/embeddings']
    subprocess.run(command)

@threaded
def download_controlnet(url):
    command = ['wget', url, '--content-disposition', '--no-verbose', '--no-clobber', '-P', '/workspace/stable-diffusion-webui/extensions/sd-webui-controlnet/models/']
    subprocess.run(command)

In [ ]:
LORAS = [
    'https://civitai.com/api/download/models/87153',
    'https://civitai.com/api/download/models/67980',
    'https://huggingface.co/naonovn/Lora/resolve/main/add_detail.safetensors',
    'https://civitai.com/api/download/models/8787',
    'https://civitai.com/api/download/models/84558',
    'https://civitai.com/api/download/models/44566',
]

VAES = [
    'https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors',
    'https://civitai.com/api/download/models/14459?type=VAE',
    'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt',
]


# https://huggingface.co/lllyasviel/ControlNet-v1-1/tree/main
CONTROL_NETS = [
    'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth',
    'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth',
    'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth',
    'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth'
]

EMBEDDINGS = [
    'https://civitai.com/api/download/models/5637',
    'https://civitai.com/api/download/models/9208',
    'https://civitai.com/api/download/models/60095',
    'https://civitai.com/api/download/models/76712',
]

In [ ]:
MODELS = [ 
    'https://civitai.com/api/download/models/15236',  # Delibarate v2
    'https://civitai.com/api/download/models/40248',  # ReV Animated
    'https://civitai.com/api/download/models/43888',  # Dreamshaper
    'https://civitai.com/api/download/models/7425',   # Counter-feit v3
    'https://civitai.com/api/download/models/69832',  # Pixar
]

In [ ]:
for mo in MODELS:
    download_model(mo)

In [ ]:
for lora in LORAS:
    download_lora(lora)

In [ ]:
for vae in VAES:
    download_vae(vae)

In [ ]:

for em in EMBEDDINGS:
    download_embedding(em)

In [ ]:
!ls -lah /workspace/stable-diffusion-webui/models/VAE

In [ ]:
# Install Controlnet
!git clone 'https://github.com/Mikubill/sd-webui-controlnet.git' '/workspace/stable-diffusion-webui/extensions/'

In [ ]:
for net in CONTROL_NETS:
    download_controlnet(net)

In [ ]:
!ls -lah /workspace/stable-diffusion-webui/models/Stable-diffusion/

In [ ]:
!mkdir /tmp/gradio/

## Deliberate v2

In [ ]:
!wget 'https://civitai.com/api/download/models/15236' -O "/workspace/stable-diffusion-webui/models/Stable-diffusion/Deliberate_v2.safetensors"

## ReV Animated

In [ ]:
!wget 'https://civitai.com/api/download/models/40248' -O "/workspace/stable-diffusion-webui/models/Stable-diffusion/revAnimated_v121.safetensors"

## DreamShaper v5.0 Baked

In [ ]:
!wget 'https://civitai.com/api/download/models/43888' -O "/workspace/stable-diffusion-webui/models/Stable-diffusion/dreamshaper_v5_vae.safetensors"

## Realistic Vision V2.0

In [ ]:
!wget 'https://civitai.com/api/download/models/29460' -O "/workspace/stable-diffusion-webui/models/Stable-diffusion/realistic_vision_v2.0.safetensors"

###  Counterfeit-V2.5

In [ ]:
!wget 'https://civitai.com/api/download/models/7425' -O "/workspace/stable-diffusion-webui/models/Stable-diffusion/Counterfeit-V25.safetensors"

## Pastel Mix

In [ ]:
!wget 'https://civitai.com/api/download/models/6297' -O "/workspace/stable-diffusion-webui/models/Stable-diffusion/Pastel-Mix.safetensors"

## Oriental-Mix

In [ ]:
!wget 'https://civitai.com/api/download/models/47998' -O "/workspace/stable-diffusion-webui/models/Stable-diffusion/Oriental-Mix.safetensors"

## LoRa

In [ ]:
!wget 'https://civitai.com/api/download/models/28907' -O "/workspace/stable-diffusion-webui/models/Lora/animeLineartMangaLike_v30MangaLike.safetensors"

In [ ]:
!wget 'https://civitai.com/api/download/models/9208' -O "/workspace/stable-diffusion-webui/models/Lora/easynegative.safetensors"

In [ ]:
!wget 'https://civitai.com/api/download/models/20068' -O "/workspace/stable-diffusion-webui/models/Lora/badhandv4.safetensors"